# MSDS 432, Assignment 4 - Quick Sort

#### Author: Brandon Moretz

In [2]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from time import time
import matplotlib.style as style

In [3]:
np.random.seed(323) # static seed so results are reproducible

In [4]:
style.use('seaborn-poster') # sets the size of the charts
style.use('ggplot') # set the theme for matplotlib

### Person Type

In [58]:
class Person:
    
    def __init__(self, first_name, last_name, address):
        """Initialize a new Person object."""
        self._fname = first_name
        self._lname = last_name
        self._address = address
        
    def first_name(self):
        return self._fname
    
    def last_name(self):
        return self._lname
    
    def address(self):
        return self._address
    
    def __str__(self):
        return self._lname + ", " + self._fname + " : " + self._address
    
    def to_dict(self):
        return {
            'address' : self._address,
            'name' : self._lname + ", " + self._fname
        }

## Sorting Algorithms

In [8]:
def quickSort(array):
    """Quicksort, uses divide and conquer strategy to sort an array, O(n log n)"""
    if len(array) < 2:
        # base case, arrays with 0 or 1 elements are already sorted.
        return array
    else:
        # recursive case
        pivot = array[0]
        # sub-array of all elements less than the pivot
        less = [i for i in array[1:] if i <= pivot]
        # sub-array of all elements greather than the pivot
        greater = [i for i in array[1:] if i >= pivot]
        return quicksort(less) + [pivot] + quicksort(greater)
    
def findSmallest(arr):
    """Finds the smallest element in an array by returning the corresponding index."""
    smallest = arr[0] # Stores the smallest value
    smallest_index = 0 # Stores the smallest index
    
    for i in range(1, len(arr)):
        if arr[i] < smallest:
            smallest = arr[i]
            smallest_index = i
    return smallest_index

def selectionSort(arr):
    """Selection Sort Algorithm, O(N**2)"""
    newArr = []
    for i in range(len(arr)):
        smallest = findSmallest(arr)
        newArr.append(arr.pop(smallest))
    return newArr

def insertionSort(arr):
    """Insertion Sort Algorithm, O(N**2)"""

    for i in range(len(arr)):
        j = i
        while j > 0 and (arr[j] <arr[j-1]):
            arr[j], arr[j-1] = arr[j-1], arr[j]
            j = j - 1
    return arr

def profile(func):
    """Utility function to help profile the runtime of code blocks."""
    start_time = time()
    func()
    return time() - start_time

### Sample Data

In [63]:
states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
characters = [chr(i) for i in range(97,123)]

def gen_code():
    return ''.join([characters[int(np.random.uniform(0, 26))] for k in range(0, 10)])

people = []

for index in range(0, 50):
    state_index = int(np.random.uniform(0, 50 - index)) # randomly select a state from the list.
    state = states[state_index] # save the value.
    states.remove(state) # remove it from possible selections.
    
    people.append(Person(gen_code(), gen_code(), state)) # add it to our list of people.

df_people = pd.DataFrame.from_records([p.to_dict() for p in people]) # convert our model of people into a pandas dataframe.

In [64]:
df_people # show the dataframe of people.

,address,name
0,NM,"zmgquxtijx, unmnhnhjyg"
1,CT,"ynvilzsdtr, ozfwssgnve"
2,MN,"eyizljflne, czegvvdewr"
3,GA,"ccyuqciacg, fvvfwaperk"
4,OR,"vphveitshv, tndbqdupse"
5,AK,"gztemzpazh, oymzzyuxet"
6,IA,"ogpqemmgyg, bobjknecuz"
7,NV,"aahyufkydm, bbovqpcaku"
8,DE,"etegukbvgj, yxxinxlgon"
9,MO,"uouekgeosq, cwfpybbecd"


## Benchmark

In [10]:
print(quicksort([10, 5, 2, 3]))

[2, 3, 5, 10]


## Results

## Summary